In [81]:
from transformers import Trainer, TrainingArguments, DistilBertForTokenClassification, DistilBertTokenizerFast, DataCollatorForTokenClassification, pipeline
from datasets import load_metric, Dataset
import numpy as np
import torch

In [82]:
tokenizer=DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [83]:
snips_file=open("Data/Snips Train Data.txt")
snips_rows=snips_file.readlines()
snips_rows[:20]

['listen O\n',
 'to O\n',
 'westbam B-artist\n',
 'alumb O\n',
 'allergic B-album\n',
 'on O\n',
 'google B-service\n',
 'music I-service\n',
 'PlayMusic\n',
 '\n',
 'add O\n',
 'step B-entity_name\n',
 'to I-entity_name\n',
 'me I-entity_name\n',
 'to O\n',
 'the O\n',
 '50 B-playlist\n',
 'clásicos I-playlist\n',
 'playlist O\n',
 'AddToPlaylist\n']

In [84]:
utterances = []
tokenized_utterances = []
labels_for_tokens = []
sequence_labels = []
utterance, tokenized_utterance, label_for_utterances = '', [], []

for snip_row in snips_rows:
    if len(snip_row) == 2:  # Skip rows with insufficient data
        continue
    if ' ' not in snip_row.strip():  # Sequence label
        sequence_label = snip_row.strip()
        if sequence_label:  # Only add non-empty labels
            sequence_labels.append(sequence_label)
        if utterance.strip():  # Avoid appending empty utterances
            utterances.append(utterance.strip())
            tokenized_utterances.append(tokenized_utterance)
            labels_for_tokens.append(label_for_utterances)
        # Reset for next sequence
        utterance = ''
        tokenized_utterance = []
        label_for_utterances = []
        continue
    token, token_label = snip_row.split(' ')
    token_label = token_label.strip()
    utterance += f'{token} '
    tokenized_utterance.append(token)
    label_for_utterances.append(token_label)

In [85]:
len(labels_for_tokens) , len(tokenized_utterances), len(utterances), len(sequence_labels)

(13084, 13084, 13084, 13084)

In [86]:
print(tokenized_utterances[0])
print(labels_for_tokens[0])
print(utterances[0])
print(sequence_labels[0])

['listen', 'to', 'westbam', 'alumb', 'allergic', 'on', 'google', 'music']
['O', 'O', 'B-artist', 'O', 'B-album', 'O', 'B-service', 'I-service']
listen to westbam alumb allergic on google music
PlayMusic


In [87]:
unique_sequence_labels=list(set(sequence_labels))
sequence_labels=[unique_sequence_labels.index(l) for l in sequence_labels]
print(len(unique_sequence_labels))
print(unique_sequence_labels)

7
['BookRestaurant', 'RateBook', 'SearchScreeningEvent', 'AddToPlaylist', 'PlayMusic', 'GetWeather', 'SearchCreativeWork']


In [88]:
from functools import reduce

unique_token_labels=list(set(reduce(lambda x, y:x+y,labels_for_tokens)))
labels_for_tokens=[[unique_token_labels.index(_) for _ in l]for l in labels_for_tokens]
print(len(unique_token_labels))

72


In [89]:
print(tokenized_utterances[0])
print(labels_for_tokens[0])
print(utterances[0])
print(sequence_labels[0])

['listen', 'to', 'westbam', 'alumb', 'allergic', 'on', 'google', 'music']
[54, 54, 22, 54, 30, 54, 65, 35]
listen to westbam alumb allergic on google music
4


In [90]:
snips_dataset=Dataset.from_dict(dict(utterance=utterances,label=sequence_labels,tokens=tokenized_utterances,token_label=labels_for_tokens))
snips_dataset=snips_dataset.train_test_split(0.2)

In [91]:
print(snips_dataset['train'][0])

{'utterance': 'i need a table for four at ten pm in dodge park', 'label': 0, 'tokens': ['i', 'need', 'a', 'table', 'for', 'four', 'at', 'ten', 'pm', 'in', 'dodge', 'park'], 'token_label': [54, 54, 54, 54, 54, 49, 54, 5, 50, 54, 57, 24]}


In [92]:
def tokenize_and_align_labels(examples):
    """
    Tokenizes the input text and aligns the labels with the wordpiece tokenization.

    Args:
        examples: A batch of examples containing 'tokens' and 'token_label'.
        tokenizer: The tokenizer to use for tokenization (e.g., a BERT tokenizer).

    Returns:
        A dictionary containing the tokenized inputs and aligned labels.
    """
    # Tokenize the inputs
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True, padding=True)

    labels = []
    for i, label in enumerate(examples["token_label"]):
        # Map tokenized words back to original word IDs
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # This maps the tokenized tokens to their original word indices
        previous_word_idx = None
        label_ids = []

        # Iterate through the word_ids to align the labels with tokens
        for word_idx in word_ids:
            if word_idx is None:  # Special tokens (CLS, SEP, etc.)
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # First token of a new word
                label_ids.append(label[word_idx])
            else:  # Other tokens in the same word
                label_ids.append(-100)  # Set to -100 for subword tokens
            previous_word_idx = word_idx
        
        labels.append(label_ids)

    # Add the aligned labels to the tokenized inputs
    tokenized_inputs["labels"] = labels
    return tokenized_inputs


In [93]:
snips_dataset['train'][0]

{'utterance': 'i need a table for four at ten pm in dodge park',
 'label': 0,
 'tokens': ['i',
  'need',
  'a',
  'table',
  'for',
  'four',
  'at',
  'ten',
  'pm',
  'in',
  'dodge',
  'park'],
 'token_label': [54, 54, 54, 54, 54, 49, 54, 5, 50, 54, 57, 24]}

In [94]:
token_clf_token_snips=snips_dataset.map(tokenize_and_align_labels,batched=True)

100%|██████████| 3/3 [00:00<00:00, 20.53ba/s]


In [95]:
token_clf_token_snips['train'][0]

{'utterance': 'i need a table for four at ten pm in dodge park',
 'label': 0,
 'tokens': ['i',
  'need',
  'a',
  'table',
  'for',
  'four',
  'at',
  'ten',
  'pm',
  'in',
  'dodge',
  'park'],
 'token_label': [54, 54, 54, 54, 54, 49, 54, 5, 50, 54, 57, 24],
 'input_ids': [101,
  1045,
  2342,
  1037,
  2795,
  2005,
  2176,
  2012,
  2702,
  7610,
  1999,
  11898,
  2380,
  102,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'labels': [-100,
  54,
  54,
  54,
  54,
  54,
  49,
  54,
  5,
  50,
  54,
  57,
  24,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100,
  -100]}

In [96]:
token_clf_token_snips['train'] = token_clf_token_snips['train'].remove_columns(['utterance', 'label', 'tokens' , 'token_label'])
token_clf_token_snips['test'] = token_clf_token_snips['test'].remove_columns(['utterance', 'label', 'tokens' , 'token_label'])
token_clf_token_snips

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 10467
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 2617
    })
})

In [97]:
token_data_collator=DataCollatorForTokenClassification(tokenizer=tokenizer)

In [101]:
token_clf_model=DistilBertForTokenClassification.from_pretrained('distilbert-base-uncased',num_labels=len(unique_token_labels))
token_clf_model.config.id2label={i: l for i,l in enumerate(unique_token_labels)}

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [102]:
token_clf_model.config.id2label[0]

'I-object_part_of_series_type'

In [103]:
epochs = 2
warmup_steps = len(token_clf_token_snips['train']) // 5
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

training_arguments = TrainingArguments(
    output_dir='./snips_token_clf/results',
    num_train_epochs=epochs,
    per_device_eval_batch_size=32,
    per_device_train_batch_size=32,
    load_best_model_at_end=True,
    warmup_steps=warmup_steps,
    weight_decay=0.05,
    logging_steps=1,
    log_level="info",
    eval_strategy='epoch',
    save_strategy='epoch',
    no_cuda=True
)

token_clf_model.to(device)
trainer=Trainer(model=token_clf_model,args=training_arguments,train_dataset=token_clf_token_snips['train'],eval_dataset=token_clf_token_snips['test'],data_collator=token_data_collator)

/opt/homebrew/Caskroom/miniconda/base/envs/pytorch_env/lib/python3.8/site-packages/transformers/training_args.py:1583: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(


In [104]:
trainer.evaluate()


***** Running Evaluation *****
  Num examples = 2617
  Batch size = 32
100%|██████████| 82/82 [00:55<00:00,  1.47it/s]


{'eval_loss': 4.29494047164917,
 'eval_model_preparation_time': 0.0015,
 'eval_runtime': 56.119,
 'eval_samples_per_second': 46.633,
 'eval_steps_per_second': 1.461}

In [105]:
trainer.train()

***** Running training *****
  Num examples = 10,467
  Num Epochs = 2
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 656
  Number of trainable parameters = 66,418,248
  0%|          | 1/656 [00:02<31:26,  2.88s/it]

{'loss': 4.3009, 'grad_norm': 5.085914611816406, 'learning_rate': 2.3889154323936934e-08, 'epoch': 0.0}


  0%|          | 2/656 [00:05<30:57,  2.84s/it]

{'loss': 4.3093, 'grad_norm': 4.624221324920654, 'learning_rate': 4.777830864787387e-08, 'epoch': 0.01}


  0%|          | 3/656 [00:08<30:17,  2.78s/it]

...


***** Running Evaluation *****
  Num examples = 2617
  Batch size = 32
                                                 
100%|██████████| 656/656 [34:42<00:00,  2.53s/it]Saving model checkpoint to ./snips_token_clf/results/checkpoint-656
Configuration saved in ./snips_token_clf/results/checkpoint-656/config.json


{'eval_loss': 0.36779090762138367, 'eval_model_preparation_time': 0.0015, 'eval_runtime': 87.5893, 'eval_samples_per_second': 29.878, 'eval_steps_per_second': 0.936, 'epoch': 2.0}


Model weights saved in ./snips_token_clf/results/checkpoint-656/model.safetensors


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ./snips_token_clf/results/checkpoint-656 (score: 0.36779090762138367).
100%|██████████| 656/656 [34:45<00:00,  3.18s/it]

{'train_runtime': 2085.8323, 'train_samples_per_second': 10.036, 'train_steps_per_second': 0.315, 'train_loss': 1.8134790027014367, 'epoch': 2.0}


TrainOutput(global_step=656, training_loss=1.8134790027014367, metrics={'train_runtime': 2085.8323, 'train_samples_per_second': 10.036, 'train_steps_per_second': 0.315, 'total_flos': 202918092294960.0, 'train_loss': 1.8134790027014367, 'epoch': 2.0})

In [106]:
trainer.evaluate()


***** Running Evaluation *****
  Num examples = 2617
  Batch size = 32
100%|██████████| 82/82 [02:24<00:00,  1.76s/it]


{'eval_loss': 0.36779090762138367,
 'eval_model_preparation_time': 0.0015,
 'eval_runtime': 146.566,
 'eval_samples_per_second': 17.855,
 'eval_steps_per_second': 0.559,
 'epoch': 2.0}

In [ ]:
pipe=pipeline('token-classification', token_clf_model,tokenizer=tokenizer)
pipe('Add Two Coins by dispatch to my road trip playlist')

In [107]:
pipe=pipeline('token-classification', token_clf_model,tokenizer=tokenizer)
pipe('Rate this playlist out of 5')

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Disabling tokenizer parallelism, we're using DataLoader multithreading already


[{'entity': 'B-object_select',
  'score': 0.48078132,
  'index': 2,
  'word': 'this',
  'start': 5,
  'end': 9},
 {'entity': 'B-best_rating',
  'score': 0.8649718,
  'index': 7,
  'word': '5',
  'start': 26,
  'end': 27}]